Let us consdider the text file Desdemona.txt that consists of the string:

Have you pray'd to-night, Desdemona?

Let us take the view of the file as the 8 x 36 binary matrix/array whose columns are the `transposed' bytes (in the context of ASCII/UTF-8 encoding) of the file. By our convention, the rows and columns of the array are indexed starting with 1. 

We can have a look at this array by running the code:

In [2]:
from bitstring import Bits


bits_of_our_file_as_bools = Bits(filename="Desdemona.txt")
for k in range(0, 8):
    bits_in_the_same_position_k = bits_of_our_file_as_bools[k::8]
    print(bits_in_the_same_position_k.bin)


000000000000000000000000000000000000
111101110111101011011111001111111110
011111111111111111111111110111111111
001001010110100010000001000010000001
100001100000100001111010100000011101
001100110000011011110101101101111101
001000100010010001010100000010001101
010101110001110001101100000110111011


The file Desdemona.txt is perfectly small for being also inspected with the help of pandas-2:

In [7]:
import pyarrow
import pandas as pd

from bitstring import Bits

indices = ["bit 1:", "bit 2:", "bit 3:", "bit 4:",
           "bit 5:", "bit 6:", "bit 7:", "bit 8:"]

bits_of_our_file_as_bools = Bits(filename="Desdemona.txt")

our_bytes_for_pd_series = dict()
counter = 0
for byte in bits_of_our_file_as_bools.cut(8):
    counter += 1
    our_bytes_for_pd_series[counter] = pd.Series(
        byte, index=indices, dtype="bool[pyarrow]")

our_df = pd.DataFrame(our_bytes_for_pd_series)

print(our_df)


           1      2      3      4      5      6      7      8      9      10   
bit 1:  False  False  False  False  False  False  False  False  False  False  \
bit 2:   True   True   True   True  False   True   True   True  False   True   
bit 3:  False   True   True   True   True   True   True   True   True   True   
bit 4:  False  False   True  False  False   True  False   True  False   True   
bit 5:   True  False  False  False  False   True   True  False  False  False   
bit 6:  False  False   True   True  False  False   True   True  False  False   
bit 7:  False  False   True  False  False  False   True  False  False  False   
bit 8:  False   True  False   True  False   True   True   True  False  False   

        ...     27     28     29     30     31     32     33     34     35   
bit 1:  ...  False  False  False  False  False  False  False  False  False  \
bit 2:  ...   True   True   True   True   True   True   True   True   True   
bit 3:  ...  False   True   True   True   Tru

All that remains to do now is to make the imaginable substitutions (0 -> 1) and (1 -> -1) for the entries of our initial array, and then 
to apply Proposition 5.9 (essentially Proposition 2.4 from the preprint https://arxiv.org/abs/1801.02601) from the monograph Symmetric Cycles, Singapore: Jenny Stanford Publishing, 2023, to the resulting rows:  

In [8]:
from bitstring import Bits
from itertools import groupby
from collections import deque


def to_deque_indices_of_vectors_in_decomp_sequence_wrt_disting_symm_cycle(bstring: Bits) -> deque[int]:

    size = len(bstring)

    # The following heavy-weight function is split below into smaller fragments.
    inclusion_maximal_intervals_of_the_negative_part = deque(map(lambda b: (b[0][1], b[-1][1]),
                                                             map(lambda p: deque(p[1]),
                                                                 filter(lambda group: group[0],
                                                                        groupby(zip(bstring, range(len(bstring))),
                                                                                lambda b_i: b_i[0])))))
    # Uncomment four lines below if you wish to split the above heavy-weight function into lighter fragments.
    #
    # b_indexed = zip(bstring, range(len(bstring)))
    # grouped = groupby(b_indexed, lambda b_i: b_i[0])
    # inclusion_maximal_blocks_of_the_negative_part = map(lambda p: deque(p[1]), filter(lambda group: group[0], grouped))
    # inclusion_maximal_intervals_of_the_negative_part = deque(map(lambda b: (b[0][1], b[-1][1]), inclusion_maximal_blocks_of_the_negative_part))

    rho = len(inclusion_maximal_intervals_of_the_negative_part)
    part_of_indices_plus, part_of_indices_minus = deque(), deque()

    # The if-elif-else fragment starting here merely automates the key Proposition 5.9
    # (essentially Proposition 2.4 from the preprint https://arxiv.org/abs/1801.02601)
    # from the monograph Symmetric Cycles, Singapore: Jenny Stanford Publishing, to appear.
    # Attention: Proposition 5.9 was formulated for {1,-1}-vectors!
    if bstring[0] and not bstring[-1]:  # see Proposition 5.9(i)
        part_of_indices_plus.append(
            inclusion_maximal_intervals_of_the_negative_part[0][1]+1)
        for p in range(2, rho + 1):
            part_of_indices_plus.append(
                inclusion_maximal_intervals_of_the_negative_part[p-1][1]+1)
            part_of_indices_minus.append(
                inclusion_maximal_intervals_of_the_negative_part[p-1][0]+size)
    elif bstring[0] and bstring[-1]:  # see Proposition 5.9(ii)
        part_of_indices_minus.append(size)
        if(rho > 1):
            part_of_indices_plus.append(
                inclusion_maximal_intervals_of_the_negative_part[0][1]+1)
        for p in range(2, rho):
            part_of_indices_plus.append(
                inclusion_maximal_intervals_of_the_negative_part[p-1][1]+1)
            part_of_indices_minus.append(
                inclusion_maximal_intervals_of_the_negative_part[p-1][0]+size)
        if(rho > 1):
            part_of_indices_minus.append(
                inclusion_maximal_intervals_of_the_negative_part[rho-1][0]+size)
    elif (not bstring[0]) and (not bstring[-1]):  # see Proposition 5.9(iii)
        part_of_indices_plus.append(0)
        for p in range(1, rho + 1):
            part_of_indices_plus.append(
                inclusion_maximal_intervals_of_the_negative_part[p-1][1]+1)
            part_of_indices_minus.append(
                inclusion_maximal_intervals_of_the_negative_part[p-1][0]+size)
    else:  # i.e., (not bstring[0]) and bstring[-1]; see Proposition 5.9(iv)
        for p in range(1, rho):
            part_of_indices_plus.append(
                inclusion_maximal_intervals_of_the_negative_part[p-1][1]+1)
            part_of_indices_minus.append(
                inclusion_maximal_intervals_of_the_negative_part[p-1][0]+size)
        part_of_indices_minus.append(
            inclusion_maximal_intervals_of_the_negative_part[rho-1][0]+size)
    return(part_of_indices_plus + part_of_indices_minus)


def linear_algebraic_portrait_8xt_of_file(filename: str) -> tuple[int, int, deque[deque]]:
    portrait = deque()
    bits_of_our_file_as_bools = Bits(filename=filename)
    for k in range(0, 8):
        bits_in_the_same_position_k = bits_of_our_file_as_bools[k::8]
        if not k:
            portrait.append(8)
            portrait.append(len(bits_in_the_same_position_k))
        portrait.append(to_deque_indices_of_vectors_in_decomp_sequence_wrt_disting_symm_cycle(
            bits_in_the_same_position_k))
    return(portrait)


### And now, an experiment follows. Cf. the portrait  (2.11)  ###
### in the preprint https://arxiv.org/abs/2303.16135          ###

print()
for item in linear_algebraic_portrait_8xt_of_file("Desdemona.txt"):
    print(item)



8
36
deque([0])
deque([4, 8, 13, 15, 18, 24, 35, 41, 45, 50, 52, 55, 62])
deque([26, 37, 63])
deque([3, 6, 8, 11, 13, 17, 24, 29, 38, 41, 43, 45, 48, 52, 59, 64, 71])
deque([1, 7, 13, 21, 23, 25, 34, 36, 41, 48, 53, 58, 60, 67, 71])
deque([4, 8, 15, 20, 22, 25, 28, 34, 38, 42, 49, 52, 57, 59, 62, 65, 71])
deque([3, 7, 11, 14, 18, 20, 22, 29, 34, 38, 42, 46, 49, 53, 55, 57, 64, 68, 71])
deque([2, 4, 8, 14, 19, 22, 29, 33, 37, 39, 41, 47, 53, 56, 63, 66, 70])
